# Defect Detection with U-Net

The project of SciRP in 2024 fall.

Model used: U-Net(with residual and attention)

Dataset: kolektorSSD

### Training Part

Using tensorboard to make the log:

In [1]:
import torch,gc
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

import utils.network as nt
from utils.trainer import Trainer
from utils.dataloader import CropDataset
from utils.loss import DiceBCELoss

gc.collect()
torch.cuda.empty_cache()

In [2]:
# hyperparameters
lr = 1e-4
epochs = 150
batch_size = 64
weight_decay = 0
retrain = 0
# the lr changing
stepsize = 15
gamma = 0.5

# create model & citerion
# model = nt.U_Net(img_ch=1, output_ch=1)
# model = nt.U_Net_(img_ch=1, output_ch=1)
# model = nt.ResU_Net(img_ch=1, output_ch=1)
# model = nt.AttU_Net(img_ch=1, output_ch=1)
model = nt.ResAttU_Net(img_ch=1, output_ch=1)
# criterion = DiceBCELoss()
criterion = nn.BCEWithLogitsLoss()

# create SummaryWriter
logdir = "log/w12/" + 'net:ResAttU ' + 'retrain:01 ' + 'loss:bce '+ 'decay:(1e-4,10,0.5) '+ 'bs:64 '
writer = SummaryWriter(logdir)

print('Setting completed!')

Setting completed!


In [3]:
# set device & parallel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_ids = list(range(torch.cuda.device_count()))
model = nn.DataParallel(model, device_ids=device_ids).to(device)
print(f'Using {len(device_ids)} devices.')

Using 2 devices.


In [4]:
# transform
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
#    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])

# data set
data_dir = './data'
dataset = CropDataset(data_dir, transform=transform)
print(f'Loaded {len(dataset)} samples.')
# divide dataset into training and testing sets
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print('Dataset completed!')

Data loaded
Found 400 images and 400 labels
Loaded 400 samples.
Dataset completed!


In [5]:
# optimizer and criterion
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
torch.optim.lr_scheduler.StepLR(optimizer, step_size=stepsize, gamma=gamma)

In [6]:
# load pre-trained model
if retrain:
    model = torch.load('model.pth')
    model.to(device)
else:
    model.to(device)

In [7]:
# create trainer and train the model
print(f'Learning rate: {lr}; Batch size: {batch_size}; Epochs: {epochs}')
trainer = Trainer(model, optimizer, criterion, device, writer)
trainer.train(train_loader, test_loader, epochs=epochs)
print('Training Finished!')

Learning rate: 0.0001; Batch size: 64; Epochs: 150


Epoch 1/150: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:12<00:00,  2.03s/it, loss=0.439]


Epoch 1/150, Train Loss: 0.4385417881939146, Validation Loss: 0.46662282943725586



Epoch 2/150: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.85s/it, loss=0.25]


Epoch 2/150, Train Loss: 0.24978770315647125, Validation Loss: 0.23170546690622967



...

Epoch 148/150, Train Loss: 0.005425840763362229, Validation Loss: 0.004201034937674801



Epoch 149/150: 100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.84s/it, loss=0.00559]


Epoch 149/150, Train Loss: 0.005592912645725948, Validation Loss: 0.0023140681247847774



Epoch 150/150: 100%|█████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.85s/it, loss=0.0053]


Epoch 150/150, Train Loss: 0.005303352306428779, Validation Loss: 0.003701078938320279

Training Finished!


In [8]:
# save the model
torch.save(model, 'model.pth')

# close the SummaryWriter
writer.close()

### Testing Part

Visualize the training result:

In [8]:
# visualize the result
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt

import utils.network as nt
from utils.dataloader import CropDataset

model = torch.load('model.pth')

/tmp/ipykernel_1466909/3521962505.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('model.pth')


In [9]:
# set device & parallel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_ids = list(range(torch.cuda.device_count()))
model = nn.DataParallel(model, device_ids=device_ids).to(device)
print(f'Using {len(device_ids)} devices.')
model.to(device)

Using 2 devices.


DataParallel(
  (module): DataParallel(
    (module): ResAttU_Net(
      (Maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (Conv1): res_conv(
        (conv): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
        )
        (res): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (Conv2): res_conv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(

In [5]:
# load the test data(all)
# transform
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
#    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])

# data set
data_dir = './data'
dataset = CropDataset(data_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=1, shuffle=False)

# load the (i+1)th item
for i, (image, label) in enumerate(train_loader):
    if i == 328:
        image, label = image[1].to(device), label[1].to(device)
        print(image.type)
        break

Data loaded
Found 400 images and 400 labels


NameError: name 'device' is not defined

In [ ]:
# forward
with torch.no_grad():
    output = model(image)
loss = criterion(output, label)
print(f'Loss: {loss.item()}')

# binary
output_binary = (output > 0.3).float()

# transform to numpy arrays
image_np = image.cpu().numpy().squeeze()
label_np = label.cpu().numpy().squeeze()
output_np = output.cpu().numpy().squeeze()
output_binary_np = output_binary.cpu().numpy().squeeze()

print(np.max(output_np))

In [ ]:
# figure
fig, axs = plt.subplots(1, 4, figsize=(20, 5))

axs[0].imshow(image_np, cmap='gray')
axs[0].set_title('Original Image')
axs[1].imshow(output_np, cmap='gray')
axs[1].set_title('Model Output')
axs[2].imshow(output_binary_np, cmap='gray')
axs[2].set_title('Binary Output')
axs[3].imshow(label_np, cmap='gray')
axs[3].set_title('Label')

for ax in axs:
    ax.axis('off')
plt.savefig('result.png')
plt.show()